In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
btc=pd.read_csv('btcusdt_1h_zelta.csv')
btc

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...,...
11517,2019-04-28 22:30:00,5320.00,5330.00,5307.20,5319.63,549.218007
11518,2019-04-28 23:30:00,5318.70,5321.03,5302.19,5308.47,536.588642
11519,2019-04-29 00:30:00,5308.12,5313.00,5259.48,5290.00,1342.535743
11520,2019-04-29 01:30:00,5291.17,5309.00,5278.49,5282.45,554.972866


In [2]:
WINDOW_SIZE=6
HORIZON=1

class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self,
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs):
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers


    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x)
    theta = self.theta_layer(x)

    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [3]:
bitcoin_prices=pd.DataFrame(btc['close'])
bitcoin_prices.rename(columns={'close':'Price'},inplace=True)
bitcoin_prices_nbeats = bitcoin_prices.copy()
for i in range(WINDOW_SIZE):
  bitcoin_prices_nbeats[f"Price+{i+1}"] = bitcoin_prices_nbeats["Price"].shift(periods=i+1)
bitcoin_prices_nbeats.dropna().head()

,Price,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
6,13780.41,13558.99,13601.01,13410.03,13330.18,13203.06,13529.01
7,13570.35,13780.41,13558.99,13601.01,13410.03,13330.18,13203.06
8,13499.99,13570.35,13780.41,13558.99,13601.01,13410.03,13330.18
9,13616.99,13499.99,13570.35,13780.41,13558.99,13601.01,13410.03
10,13570.01,13616.99,13499.99,13570.35,13780.41,13558.99,13601.01


In [4]:
bitcoin_prices_nbeats.max()

Price      17069.79
Price+1    17069.79
Price+2    17069.79
Price+3    17069.79
Price+4    17069.79
Price+5    17069.79
Price+6    17069.79
dtype: float64

In [39]:
X = bitcoin_prices_nbeats.dropna().drop("Price", axis=1)
X = X/bitcoin_prices_nbeats.dropna().drop("Price", axis=1).max()
y = bitcoin_prices_nbeats.dropna()["Price"]

split_size = int(len(X) * 0.85)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(9788, 9788, 1728, 1728)

In [40]:
X

,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
6,0.794327,0.796788,0.785600,0.780922,0.773475,0.792570
7,0.807298,0.794327,0.796788,0.785600,0.780922,0.773475
8,0.794992,0.807298,0.794327,0.796788,0.785600,0.780922
9,0.790870,0.794992,0.807298,0.794327,0.796788,0.785600
10,0.797725,0.790870,0.794992,0.807298,0.794327,0.796788
...,...,...,...,...,...,...
11517,0.311662,0.312463,0.310280,0.309764,0.311689,0.312527
11518,0.311640,0.311662,0.312463,0.310280,0.309764,0.311689
11519,0.310986,0.311640,0.311662,0.312463,0.310280,0.309764
11520,0.309904,0.310986,0.311640,0.311662,0.312463,0.310280


In [41]:
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

features_dataset = tf.data.Dataset.from_tensor_slices(X)
labels_dataset = tf.data.Dataset.from_tensor_slices(y)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)


train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))


BATCH_SIZE = 1024
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
features_dataset = features_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
labels_dataset = labels_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


train_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [42]:


N_EPOCHS = 5000
N_NEURONS = 512
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(6, 7)

In [43]:
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")


stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")


backcast, forecast = nbeats_block_layer(stack_input)

residuals = layers.subtract([stack_input, backcast], name=f"subtract_00")


for i, _ in enumerate(range(N_STACKS-1)):
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals)

  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")


model_7 = tf.keras.Model(inputs=stack_input,
                         outputs=forecast,
                         name="model_7_N-BEATS")


model_7.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])


model_7.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=2,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=600, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=400, verbose=1)])

Epoch 1/5000
10/10 - 45s - loss: 6845.5610 - mae: 6845.5610 - mse: 53415280.0000 - val_loss: 4376.2559 - val_mae: 4376.2559 - val_mse: 19539208.0000 - lr: 0.0010 - 45s/epoch - 4s/step
Epoch 2/5000
10/10 - 1s - loss: 7035.5742 - mae: 7035.5742 - mse: 56425988.0000 - val_loss: 989.8661 - val_mae: 989.8661 - val_mse: 1050260.5000 - lr: 0.0010 - 929ms/epoch - 93ms/step
Epoch 3/5000
10/10 - 1s - loss: 10729.7529 - mae: 10729.7529 - mse: 393831520.0000 - val_loss: 4363.3848 - val_mae: 4363.3848 - val_mse: 19426146.0000 - lr: 0.0010 - 620ms/epoch - 62ms/step
Epoch 4/5000
10/10 - 1s - loss: 7069.9771 - mae: 7069.9771 - mse: 56579796.0000 - val_loss: 4299.9072 - val_mae: 4299.9072 - val_mse: 18868668.0000 - lr: 0.0010 - 628ms/epoch - 63ms/step
Epoch 5/5000
10/10 - 1s - loss: 7507.0723 - mae: 7507.0723 - mse: 109759592.0000 - val_loss: 4226.2686 - val_mae: 4226.2686 - val_mse: 18232812.0000 - lr: 0.0010 - 615ms/epoch - 61ms/step
Epoch 6/5000
10/10 - 1s - loss: 7018.1816 - mae: 7018.1816 - mse: 5

In [44]:
def make_pred(model,input_data):
    pseudo_forecast=model.predict(input_data)
    return tf.squeeze(pseudo_forecast)


In [45]:
model_7_preds=make_pred(model_7,features_dataset)

12/12 [==============================] - 1s 17ms/step


In [46]:
def evaluate_model(y_true,y_pred):
    y_true=np.array(y_true)
    y_pred=np.array(y_pred)
    mae=tf.keras.metrics.mean_absolute_error(y_true,y_pred)
    mse=tf.keras.metrics.mean_squared_error(y_true,y_pred)
    rmse=np.sqrt(mse.numpy())
    mape=tf.keras.metrics.mean_absolute_percentage_error(y_true,y_pred)
    mase=mean_absolute_scaled_error(y_true,y_pred)
    return{'mae':mae.numpy(),
           'mse':mse.numpy(),
           'rmse':rmse,
           'mape':mape.numpy(),
           'mase':mase.numpy()}



In [16]:
# weights = model_7.get_weights()

In [30]:
# import pickle

In [37]:
# file_path = r'C:\Users\karth\All about ML\GC\worked\weights.pkl'
# with open(file_path,'wb') as file:
#   pickle.dump(weights,file)

In [47]:
import tensorflow as tf
def mean_absolute_scaled_error(y_true,y_pred):
    mae=tf.reduce_mean(tf.abs(np.array(y_true)-np.array(y_pred)))
    mae_naive_noseason=tf.reduce_mean(tf.abs(np.array(y_true[1:])-np.array(y_true[:-1])))
    return mae/mae_naive_noseason

In [48]:
import numpy as np
evaluate_model(y_true=y,y_pred=model_7_preds)

{'mae': 65.020935,
 'mse': 13996.079,
 'rmse': 118.30502,
 'mape': 0.8550134,
 'mase': 1.6141675996675378}

In [ ]:
pred=pd.Series(model_7_preds,index=btc[6:].index,name='pred_close')

In [ ]:
pred_nbeats=pd.concat([btc,pred],axis=1)

In [ ]:
pred_nbeats.to_csv('pred_nbeatsg.csv',index=False)